# Blue Brain BioExplorer
![](../bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer
from mediamaker import MovieMaker
import os

sequence_name = 'high_glucose'

hostname = 'localhost:5000'
resource_folder = '../../tests/test_files/'
    
be = BioExplorer(hostname)
core = be.core_api()
mm = MovieMaker(core)
bio_explorer_version = be.version()
print(bio_explorer_version)
be.reset()

output_folder = resource_folder + '/movies/frames/' + \
    bio_explorer_version + '/' + sequence_name + '/stills/'
print(output_folder)

## Data loading

In [ ]:
cache_folder = resource_folder + 'caches/' + bio_explorer_version + '/'
print(cache_folder)

In [ ]:
cache_filename = cache_folder + sequence_name + '_scenario.bioexplorer'
print('Loading ' + cache_filename)
dummy = core.add_model(cache_filename)

In [ ]:
nb_models = len(core.scene.models)
if nb_models != 174:
    print('ERROR: Not all models have been loaded!!! Expected 174, found ' + str(nb_models))
else:
    print('All good!')

## Rendering settings

In [ ]:
status = core.set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_samples = 3
params.gi_weight = 0.6
params.gi_distance = 100
params.shadows = 0.8
params.soft_shadows = 1.0
params.soft_shadows_samples = 1
params.fog_start = 1300
params.fog_thickness = 1300
params.max_bounces = 1
status = core.set_renderer_params(params)

## Image settings

In [ ]:
image_size = [3840, 2160]
image_samples_per_pixel = 64

command_line='mkdir -p ' + output_folder
os.system(command_line)
command_line='ls ' + output_folder
print(output_folder)
if os.system(command_line) != 0:
    print('ERROR: Failed to create output folder')

## Snapshot

In [ ]:
def set_all_materials_to_grey():
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)
        palette = list()
        for p in range(nb_materials):
            palette.append([0.25,0.25,0.25])
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids, 
            diffuse_colors=palette, specular_colors=palette)

In [ ]:
def apply_glucose_colors():
    import seaborn as sns
    shading_mode = be.SHADING_MODE_BASIC
    specular_exponent = 50
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)

        if be.NAME_GLUCOSE in model_name and 'Surfactant' in model_name:
            print(model_name)
            palette = sns.color_palette('OrRd', nb_materials)
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)
        if be.NAME_LACTOFERRIN in model_name or \
           be.NAME_SURFACTANT_HEAD in model_name or \
           be.NAME_DEFENSIN in model_name:
            print(model_name)
            palette = list()
            for p in range(nb_materials):
                palette.append([0.25,0.25,0.25])
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids,
                palette=palette)


### Frame 1100 - Surfactant head

In [ ]:
status = core.set_camera(
    orientation=[-0.02459763421881927, -0.1396917585193262, -0.003471202998622925, 0.9898833869412891],
    position=[38.74944305419922, 35.228843688964844, 5.536177158355713],
    target=[39.02582931518555, 35.179176330566406, 4.576415061950684]
)

In [ ]:
set_all_materials_to_grey()
apply_glucose_colors()

In [ ]:
mm.create_snapshot(
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/01100.png', exportIntermediateFrames=False)